In [20]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

1) Scrape the Location Counts
Use the beautifulsoup library to scrape the data (from menuism.com like the link above) on state names and corresponding number of store locations, for the following chains:
Starbucks

Dunkin’ Donuts

Peet’s Coffee & Tea

Tim Horton’s

Panera Bread

Caribou Coffee

Au Bon Pain

The Coffee Bean & Tea Leaf

McDonald’s

In [8]:
response = requests.get("https://www.menuism.com/restaurant-locations/starbucks-coffee-39564")

# Check if the request was successful
if response.status_code == 200:
    page_content = response.content
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Failed to retrieve the webpage. Status code: 502


In [9]:
soup = BeautifulSoup(page_content, "html.parser")

# Dictionaries to store the results
store_data = {}
state_abbreviations = {}

# Extracting the name of the store
store_name = soup.find("h1").find("span", {"itemprop" : "name"})

# Finding all anchor tags with titles ending with ', us'
store_tags = soup.find_all("a", title=lambda t: t and t.endswith(", us"))

for tag in store_tags:
    state_abbr = tag.get("href")[-2:].upper()
    words = tag.get_text().split("")
    exclude_words = store_name.split("")

    # Constructing the state name
    state_name = " ".join(w for w in words if w not in exclude_words and w != "locations" and not w.startswith("("))

    # Extracting the number of locations
    location_count = words[-1][1:-1]

    # Updating the dictionaries
    store_data[state_name] = location_count
    state_abbreviations[state_name] = state_abbr

NameError: name 'page_content' is not defined

2) Write a function stateabb() that takes a state name (assume it’s spelled correctly) and converts it to its state abbreviation. This can be a very simple function.

In [ ]:
def stateabb(state):
  if state == "Mississippi":
    return "MS"

  return state_name[state]

3) Parse, merge and tidy your data so that you have a row for each state and two columns: state abbrevation, location count.

In [10]:
# Creating a DataFrame from the store data dictionary
store_df = pd.DataFrame(list(store_data.items()), columns=["State", "Locations"])

# Applying the state abbreviation transformation function
store_df["State"] = store_df["State"].apply(state_name)

# Displaying the results
print("Store Name:", store_name)

print(store_df.to_string(index=False))

NameError: name 'store_data' is not defined

Supplemental Data

4) Scrape the state names and populations from this wikipedia page. Convert the state names to abbreviations and merge these data with your coffee dataset.


In [11]:
url = "https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population"

response = requests.get(url)


if response.status_code == 200:
    page_content = response.content
    soup = BeautifulSoup(page_content, "html.parser")
    population_table = soup.find("table")

    # Dictionaries to store the results
    state_populations = {}

    # Iterate through the rows and extract data
    for row in population_table.find_all("tr")[1:]:  # Skipping the header row
        columns = row.find_all("td")
        if len(columns) > 2:  # Ensure there's at least three columns
            state_name = columns[2].get_text(strip=True)
            population = columns[3].get_text(strip=True)
            state_populations[state_name] = population

    # Removing entries that are not states
    for non_state in ["Puerto Rico", "Guam", "U.S. Virgin Islands", "American Samoa", "Northern Mariana Islands",
                      "Contiguous United States", "The fifty states", "Fifty states + D.C.",
                      "Total U.S. (including D.C. and territories)"]:
        state_populations.pop(non_state, None)
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


5) Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these into R). Merge these values into your big dataset. Note: these values may be repeated for each state.


In [12]:
url = "https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=SBUX&interval=5min&apikey=XFYGY6J374NSETZP"
r = requests.get(url)
data = r.json()

print(data)


{'symbol': 'SBUX', 'annualReports': [{'fiscalDateEnding': '2022-09-30', 'reportedCurrency': 'USD', 'grossProfit': '8371100000', 'totalRevenue': '32250300000', 'costOfRevenue': '23879200000', 'costofGoodsAndServicesSold': '23879200000', 'operatingIncome': '4617800000', 'sellingGeneralAndAdministrative': '2032000000', 'researchAndDevelopment': 'None', 'operatingExpenses': '3941400000', 'investmentIncomeNet': 'None', 'netInterestIncome': '-385900000', 'interestIncome': '97000000', 'interestExpense': '482900000', 'nonInterestIncome': 'None', 'otherNonOperatingIncome': '97000000', 'depreciation': 'None', 'depreciationAndAmortization': '1447900000', 'incomeBeforeTax': '4230100000', 'incomeTaxExpense': '948500000', 'interestAndDebtExpense': 'None', 'netIncomeFromContinuingOperations': '3283400000', 'comprehensiveIncomeNetOfTax': '2671200000', 'ebit': '4713000000', 'ebitda': '6160900000', 'netIncome': '3281600000'}, {'fiscalDateEnding': '2021-09-30', 'reportedCurrency': 'USD', 'grossProfit': '

In [13]:
Revenue = data["annualReports"][0]["totalRevenue"]
print(Revenue)

32250300000


6) Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

In [14]:
def categorize_region(state):
    northeast_states = ["Connecticut", "Maine", "Massachusetts", "New Hampshire", "Vermont", "Rhode Island", "New Jersey", "New York", "Pennsylvania"]
    midwest_states = ["Illinois", "Indiana", "Michigan", "Ohio", "Wisconsin", "Iowa", "Kansas", "Minnesota", "Missouri", "Nebraska", "North Dakota", "South Dakota"]
    west_states = ["Arizona", "Colorado", "Idaho", "Montana", "Nevada", "New Mexico", "Utah", "Wyoming", "Alaska", "California", "Hawaii", "Oregon", "Washington"]
    if state in northeast_states:
        return "Northeast"
    elif state in midwest_states:
        return "Midwest"
    elif state in west_states:
        return "West"
    else:
        return "South"

In [15]:
# Creating a DataFrame from the state populations dictionary
population_df = pd.DataFrame(list(state_populations.items()), columns=["State", "Population"])

# Applying the region categorization function
population_df["Region"] = population_df["State"].apply(categorize_region)

# Cleaning and transforming state names
population_df["State"] = population_df["State"].str.strip()
population_df["State"] = population_df["State"].replace(state_abbreviations)

NameError: name 'state_abbreviations' is not defined

In [16]:
# Merging the store locations DataFrame with the state populations DataFrame
merged_df = pd.merge(store_df, population_df, on = "State", how = "inner")

# Displaying the results
print(merged_df)


NameError: name 'store_df' is not defined

7) Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):
Are some of these chains more prevalent in certain states than others? Possibly despite having less stores overall? Same questions for regions instead of states.

How does your chosen financial metric change by state and region for each chain? For example, having 5 stores in California is very different from having 5 stores in Wyoming.

Does the distribution of each chain’s stores match population distribution, by both state/region?

Do the financial data match what you’d expect based on the number and locations of the stores? Why or why not?

In [17]:
store_urls = [
    "https://www.menuism.com/restaurant-locations/starbucks-coffee-39564",
    "https://www.menuism.com/restaurant-locations/dunkin-donuts-18162",
    "https://www.menuism.com/restaurant-locations/peets-coffee-tea-84051",
    "https://www.menuism.com/restaurant-locations/tim-hortons-190025",
    "https://www.menuism.com/restaurant-locations/panera-bread-4258",
    "https://www.menuism.com/restaurant-locations/caribou-coffee-164861",
    "https://www.menuism.com/restaurant-locations/au-bon-pain-69342",
    "https://www.menuism.com/restaurant-locations/the-coffee-bean-tea-leaf-165988",
    "https://www.menuism.com/restaurant-locations/mcdonalds-21019"
]

def scrape_store_locations(url):
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.content, "html.parser")

    # Extracting the name of the store
    brand_name = soup.find("h1").find("span", {"itemprop": "name"}).get_text()

    # Dictionaries to store the results
    location_data = {}
    state_codes = {}

    # Finding all anchor tags with titles ending with ', us'
    location_tags = soup.find_all("a", title=lambda title: title and title.endswith(", us"))

    for tag in location_tags:
        state_abbr = tag.get("href")[-2:].upper()
        words = tag.get_text().split(" ")
        exclude_words = brand_name.split(" ")

        # Constructing the state name
        state_name = ' '.join(w for w in words if w not in exclude_words and w != "locations" and not w.startswith("("))

        # Extracting the number of locations
        number_of_locations = words[-1][1:-1]

        # Updating the dictionaries
        location_data[state_name] = number_of_locations
        state_codes[state_name] = state_abbr

    # Creating a DataFrame from the location data
    locations_df = pd.DataFrame(list(location_data.items()), columns=["State", "Locations"])
    locations_df["State"] = locations_df["State"].map(state_codes)
    locations_df["Brand"] = brand_name

    return locations_df

# Scraping store locations for each URL
store_dataframes = [scrape_store_locations(url) for url in store_urls]

Failed to retrieve the webpage. Status code: 502
Failed to retrieve the webpage. Status code: 502
Failed to retrieve the webpage. Status code: 502
Failed to retrieve the webpage. Status code: 502
Failed to retrieve the webpage. Status code: 502
Failed to retrieve the webpage. Status code: 502
Failed to retrieve the webpage. Status code: 502
Failed to retrieve the webpage. Status code: 502
Failed to retrieve the webpage. Status code: 502


8) Convert your code for Exercises 1-3 above to a function that takes a single argument: the menuism URL. This function should

Scrape the information on state names and corresponding number of store locations on the webpage specified

Extract the name of the company from either the URL specified or the webpage

Return a clean, organized dataset that has three columns: state abbreviation, location count, company name. Company name will likely be repeated on every row.

In [18]:
scrape_website("https://www.menuism.com/restaurant-locations/the-coffee-bean-tea-leaf-165988")

NameError: name 'scrape_website' is not defined